### Download the model weights from original Darket's release

# TODO:
1. remove hardcoded filenames from `train_aml.py` and `train.py`
2. cleanup `model_data` folder to reduce size
3. upload `*.weights` to model registry to reduce size for aml/staging

In [3]:
!pip install keras==2.2.4 tensorflow pillow matplotlib h5py tensorboard azureml-sdk onnxruntime==1.4.0 onnx==1.7.0

  Using cached Keras-2.2.4-py2.py3-none-any.whl (312 kB)


ERROR: Could not find a version that satisfies the requirement onnxruntime==1.4.0 (from versions: none)
ERROR: No matching distribution found for onnxruntime==1.4.0
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


  Using cached azureml_sdk-1.13.0-py3-none-any.whl (4.4 kB)


In [ ]:
!pip install git+git://github.com/microsoft/onnxconverter-common.git@3451bbffe61a2591a17f4d99a405b48e9ae8e395
    
!pip install git+git://github.com/onnx/keras-onnx.git@ff17787c393e2ce34d43185447d7354525f3ba87

In [ ]:
# setting the model variables

yolo3_keras_model = 'model_data/tiny_yolo_weights.h5' # need to pass this as parameter to train.py
yolo3_weights_filename = 'model_data/yolo_v3-tiny.weights' # remove this from model_data after the keras conversion
yolo3_config = 'model_data/yolov3-tiny.cfg'
model_url = 'https://pjreddie.com/media/files/yolov3-tiny.weights'


In [ ]:
# download the model
import urllib.request

urllib.request.urlretrieve(model_url, yolo3_weights_filename)

### Convert the Darket model to Keras

In [ ]:
# execute the pre-built conversion script provided in the sample
!python3  .src/convert.py $yolo3_config $yolo3_weights_filename $yolo3_keras_model

### Training data
This sample uses the [VOC Pascal dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/#devkit) referred to as _VOCDevkit_.

To generate your own annotation file and class names file.

- One row for one image;  
- Row format: `image_file_path box1 box2 ... boxN`;  
- Box format: `x_min,y_min,x_max,y_max,class_id` (no space).  
- For VOC dataset, try `python voc_annotation.py`  
Here is an example:

        ```
        path/to/img1.jpg 50,100,150,200,0 30,50,200,120,3
        path/to/img2.jpg 120,300,250,600,2
        ...
        ```

In [ ]:
# setup the folder for the training dataset

import os

dataset_folder = 'training_dataset'
dataset_filename = dataset_folder + '/VOCtrainval2007.tar'
optional_dataset_filename = dataset_folder + '/VOCtest2007.tar'

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

In [ ]:
dataset_url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar'
urllib.request.urlretrieve(dataset_url, dataset_filename)

In [ ]:
optional_dataset_url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar'
urllib.request.urlretrieve(optional_dataset_url, optional_dataset_filename)

Unpack the dataset and prepare to upload to Datastore in AML.

_This step will take a few minutes._

In [ ]:
! tar xf $dataset_filename
! tar xf $optional_dataset_filename
! python ./voc_annotation.py

In [ ]:
#Initialize Workspace 
from azureml.core import Workspace

## existing AML Workspace in config.json
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

def_blob_store = ws.get_default_datastore()

#### OR create a new workspace with the following steps

```
from azureml.core import Workspace

### Change this cell from markdown to code and run this if you need to create a workspace 
### Update the values for your workspace below
ws=Workspace.create(subscription_id="<subscription-id goes here>",
                resource_group="<resource group goes here>",
                name="<name of the AML workspace>",
                location="<location>")
                
ws.write_config()
```

Upload the VOCdevkit to the workspace datastore

In [ ]:
def_blob_store.upload("VOCdevkit", target_path="/data/VOCdevkit")

### Train the Keras model.

In [ ]:
import os
import shutil
import glob

#set the project folder
PROJECT_FOLDER = "./aml/staging"
if os.path.exists(PROJECT_FOLDER):
    shutil.rmtree(PROJECT_FOLDER)

os.makedirs(PROJECT_FOLDER, exist_ok=True)

# copy all python scripts to project folder
files = glob.glob("*.py")
for f in files:
    shutil.copy(f, PROJECT_FOLDER)

# copy all config files to the project folder
files = glob.glob("*.cfg")
for f in files:
    shutil.copy(f, PROJECT_FOLDER)

# copy all text files to the project folder
files = glob.glob("*.txt")
for f in files:
    shutil.copy(f, PROJECT_FOLDER)


In [ ]:
! cp -rf model_data $PROJECT_FOLDER # copy the dataset under the project folder

!cp -rf yolo3 $PROJECT_FOLDER # copy the model files under the project folder

In [ ]:
from azureml.data.data_reference import DataReference
from azureml.core import Dataset

training_dataset = Dataset.File.from_files(path=(def_blob_store, '/data/VOCdevkit'))

Add the optional packages and setup the training environment

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

cd = CondaDependencies.create(pip_packages=['keras==2.1.5', 'tensorflow==1.6.0', 'pillow', 'matplotlib', 'h5py', 'tensorboard'], conda_packages=['python=3.6.11'])
myenv = Environment("yolov3")
myenv.python.conda_dependencies = cd
myenv.python.conda_dependencies.add_pip_package("azureml-sdk")
myenv.python.conda_dependencies.add_channel("conda-forge")
myenv.docker.enabled = True
myenv.docker.base_image = DEFAULT_GPU_IMAGE

Setup the training compute

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
CLUSTER_NAME = "gpu-taining"

# Verify that cluster does not exist already
try:
    aml_cluster = AmlCompute(workspace=ws, name=CLUSTER_NAME)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    print("provisioning new compute target")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC6", max_nodes=8, vm_priority="lowpriority"
    )
    aml_cluster = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)

aml_cluster.wait_for_completion(show_output=True)


Define the run config for the experiment

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(
    source_directory=PROJECT_FOLDER,
    script='train.py',
    arguments=["--data", training_dataset.as_named_input('input').as_mount()],
    )

src.run_config.framework = 'python'
src.run_config.target = aml_cluster.name

# Set environment
src.run_config.environment = myenv

Run the training experiment

In [ ]:
from azureml.core import Experiment

EXPERIMENT_NAME = "keras-yolo3"

experiment = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run = experiment.submit(config=src)

In [ ]:
%%time

run.wait_for_completion(show_output=True)

In [ ]:
# register the model in the model registry

my_trained_model = 'tiny_yolov3'

from azureml.core import Model
model = Model(ws, my_trained_model)

# download the trainied model
model.download(target_dir='model_data', exist_ok=True)

### Convert the model to ONNX and register in the model registry

In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow==1.15

In [ ]:
!pip install tf2onnx

In [ ]:
# install the Keras-to-ONNX converter dependent packages.
# Installing from master branch to get the latest changes.
!pip install git+https://github.com/onnx/keras-onnx.git@master 
!pip install git+https://github.com/microsoft/onnxconverter-common.git@master 

In [ ]:
    # parser.add_argument('model_path', help='Path to Keras weights file.')
    # parser.add_argument('anchors_path', help='Path to model achors file file.')
    # parser.add_argument('onnx_filename', help='Path to converted ONNX model')
    # parser.add_argument('target_opset', help='ONNX opset version')

model_onnx = 'yolov3-tiny.onnx'

!python3 ./convert2onnx.py 'model_data/tiny_yolo_weights.h5' 'model_data/tiny_yolo_anchors.txt' $model_onnx 11
#!python3 ./convert2onnx.py 'model_data/trained_weights_final.h5' 'model_data/tiny_yolo_anchors.txt' $model_onnx 11

In [ ]:
# Register the ONNX model in the workspace registry
from azureml.core.model import Model
Model.register(model_path = model_onnx, 
                model_name = "yolo3.onnx", 
                workspace = ws,
                description="ONNX model converted from trained Keras/Tf")

# CLEANUP!!!

In [ ]:
shutil.rmtree(PROJECT_FOLDER)

## FINISHED. 
Transition to the ADO to review status of the pipeline.